# 생성한 데이터셋 업로드

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


In [4]:
import pandas as pd

df = pd.read_csv("data/ragas_synthetic_dataset.csv")
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the key considerations regarding AI f...,['∙ 3개국은 기반모델 전반에 대한 규제가 기술 중립적이고 위험 기반의 AI 규제...,Key considerations regarding AI foundation mod...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key features and improvements of ...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,The key features and improvements of the LLM '...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What is the purpose of the AAAI Conference?,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,The purpose of the AAAI Conference is to advan...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What role does content moderation play in the ...,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What FTC concerns exist on AI comp. & consumer...,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 연방거래...,The FTC has raised concerns regarding AI-relat...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


## DeepL 을 활용한 번역

- [DeepL API 키 발급](https://www.deepl.com/ko/your-account/keys)

발급 받은 키를 `.env` 파일에 `DEEPL_API_KEY` 로 저장합니다.

In [13]:
# !pip install -qU deepl

In [5]:
import os
from langchain_teddynote.translate import Translator

# api키 설정
deepl_api_key = os.getenv("DEEPL_API_KEY")

# 객체 생성
translator = Translator(deepl_api_key, "EN", "KO")

# 번역 실행
translated_text = translator("hello, nice to meet you")
print(translated_text)

안녕하세요, 만나서 반가워요


In [6]:
from tqdm import tqdm

# 번역
for i, row in tqdm(df.iterrows(), total=len(df), desc="번역 진행 중"):
    df.loc[i, "question_translated"] = translator(row["question"])
    df.loc[i, "ground_truth_translated"] = translator(row["ground_truth"])

번역 진행 중: 100%|██████████| 10/10 [00:09<00:00,  1.05it/s]


번역 전

In [11]:
# question, ground_truth 열을 확인합니다.
df.loc[:, ["question", "ground_truth"]].head()

,question,ground_truth
0,What are the key considerations regarding AI f...,Key considerations regarding AI foundation mod...
1,What are the key features and improvements of ...,The key features and improvements of the LLM '...
2,What is the purpose of the AAAI Conference?,The purpose of the AAAI Conference is to advan...
3,What role does content moderation play in the ...,Content moderation plays a crucial role in AI ...
4,What FTC concerns exist on AI comp. & consumer...,The FTC has raised concerns regarding AI-relat...


번역한 결과를 확인합니다.

In [12]:
# question_translated, ground_truth_translated 열을 확인합니다.
df.loc[:, ["question_translated", "ground_truth_translated"]].head()

,question_translated,ground_truth_translated
0,EU 규정의 맥락에서 AI 기반 모델과 관련하여 주요 고려 사항은 무엇인가요?,EU 규정의 맥락에서 AI 기반 모델과 관련된 주요 고려 사항으로는 의무적 자율 규...
1,"성능 지표, 아키텍처 설계 및 애플리케이션 영역의 변경 사항을 포함하여 이전 버전과...",이전 버전과 비교한 LLM 'Tongyi Qianwen 2.0'의 주요 기능 및 개...
2,AAAI 컨퍼런스의 목적은 무엇인가요?,"AAAI 컨퍼런스의 목적은 연구자와 실무자가 모여 인공지능의 응용, 방법론, 미래 ..."
3,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
4,AI 보상 및 소비자 보호에 대한 FTC의 우려는 무엇인가요?,FTC는 AI 관련 경쟁 및 소비자 보호 문제에 대한 우려를 제기했습니다. 특히 A...


In [13]:
# question, ground_truth 열을 삭제하고 question_translated, ground_truth_translated 열의 이름을 변경합니다.
df.drop(columns=["question", "ground_truth"], inplace=True)
df.rename(
    columns={
        "question_translated": "question",
        "ground_truth_translated": "ground_truth",
    },
    inplace=True,
)
df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['∙ 3개국은 기반모델 전반에 대한 규제가 기술 중립적이고 위험 기반의 AI 규제...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,EU 규정의 맥락에서 AI 기반 모델과 관련하여 주요 고려 사항은 무엇인가요?,EU 규정의 맥락에서 AI 기반 모델과 관련된 주요 고려 사항으로는 의무적 자율 규...
1,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,"성능 지표, 아키텍처 설계 및 애플리케이션 영역의 변경 사항을 포함하여 이전 버전과...",이전 버전과 비교한 LLM 'Tongyi Qianwen 2.0'의 주요 기능 및 개...
2,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AAAI 컨퍼런스의 목적은 무엇인가요?,"AAAI 컨퍼런스의 목적은 연구자와 실무자가 모여 인공지능의 응용, 방법론, 미래 ..."
3,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
4,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 연방거래...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 보상 및 소비자 보호에 대한 FTC의 우려는 무엇인가요?,FTC는 AI 관련 경쟁 및 소비자 보호 문제에 대한 우려를 제기했습니다. 특히 A...


In [14]:
# 번역한 데이터셋을 저장합니다.
df.to_csv("data/ragas_synthetic_dataset_translated.csv", index=False)

## Hugging Face Dataset 업로드

Pandas DataFrame 을 Hugging Face Dataset 으로 변환하여 업로드를 진행합니다.

In [15]:
from datasets import Dataset

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 확인
print(dataset)

Dataset({
    features: ['contexts', 'evolution_type', 'metadata', 'episode_done', 'question', 'ground_truth'],
    num_rows: 10
})


In [ ]:
from datasets import Dataset
import os

# pandas DataFrame을 Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

# 데이터셋 이름 설정 (원하는 이름으로 변경하세요)
dataset_name = "teddylee777/rag-synthetic-dataset"

# 데이터셋 업로드
dataset.push_to_hub(
    dataset_name,
    private=True,  # private=False로 설정하면 공개 데이터셋이 됩니다.
    split="korean_v1",  # 데이터셋 split 이름 입력
    token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
)

[참고] Dataset Viewer 는 표기되기까지 시간이 좀 걸릴 수 있습니다.